In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 32

class Autoencoder(nn.Module):
    def __init__(self, input_size=128):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x)), negative_slope=0.1)
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             320
       BatchNorm2d-2           [-1, 32, 32, 32]              64
            Conv2d-3           [-1, 64, 16, 16]          18,496
       BatchNorm2d-4           [-1, 64, 16, 16]             128
   ConvTranspose2d-5             [-1, 32, 8, 8]          18,464
       BatchNorm2d-6             [-1, 32, 8, 8]              64
   ConvTranspose2d-7           [-1, 16, 16, 16]           4,624
       BatchNorm2d-8           [-1, 16, 16, 16]              32
   ConvTranspose2d-9            [-1, 2, 16, 16]             290
Total params: 42,482
Trainable params: 42,482
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.85
Params size (MB): 0.16
Estimated Total Size (MB): 1.01
---------------------------------------------

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00315045 (0.02117042), PSNR 25.01627731 (22.11995434), SSIM 0.63509083 (0.48891963)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00437547 (0.00394613), PSNR 23.58974838 (24.08890400), SSIM 0.52243334 (0.60788565)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00340168 (0.00344520), PSNR 24.68307114 (24.67271574), SSIM 0.68435550 (0.65068087)
Finished training epoch 1
Validate: MSE 0.00330877 (0.00302422), PSNR 24.80332756 (25.25913708), SSIM 0.60871285 (0.68618539)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00286918 (0.00307891), PSNR 25.42241669 (25.14356360), SSIM 0.72055376 (0.69935231)
Finished training epoch 2
Validate: MSE 0.00350616 (0.00289355), PSNR 24.55168152 (25.44712007), SSIM 0.64131188 (0.71772257)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00311356 (0.00289551), PSNR 25.06743050 (25.40845217), SSIM 0.74609733 (0.72377208)
Finished training epoch 3
Validate: MSE 0.00323943 (0.00281485), PSNR 24.89530945 (25.56781487), SSIM 0.65688932 (0.73106552)
Finished validation.
Starting training epoch 4
Epoch: 4, MSE 0.00294496 (0.002831

Validate: MSE 0.00287641 (0.00246769), PSNR 25.41148567 (26.15055984), SSIM 0.71334922 (0.77557334)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00199344 (0.00235579), PSNR 27.00396347 (26.30097790), SSIM 0.78818023 (0.77846746)
Finished training epoch 31
Validate: MSE 0.00278004 (0.00246376), PSNR 25.55948257 (26.16053533), SSIM 0.71668249 (0.77824102)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00209834 (0.00236424), PSNR 26.78124046 (26.28341871), SSIM 0.76762486 (0.77850346)
Finished training epoch 32
Validate: MSE 0.00274932 (0.00268824), PSNR 25.60774040 (25.81536731), SSIM 0.71477699 (0.76835218)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00238458 (0.00236082), PSNR 26.22588539 (26.29276777), SSIM 0.78065914 (0.77876922)
Finished training epoch 33
Validate: MSE 0.00277403 (0.00245871), PSNR 25.56889534 (26.16273794), SSIM 0.71253151 (0.77606118)
Finished validation.
Starting training epoch 34
Epoch: 34, MSE 0.0022656

Validate: MSE 0.00282195 (0.00249731), PSNR 25.49450302 (26.10060195), SSIM 0.71018779 (0.77438119)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00245552 (0.00227578), PSNR 26.09857178 (26.45185885), SSIM 0.77006924 (0.78005848)
Finished training epoch 61
Validate: MSE 0.00352425 (0.00358870), PSNR 24.52933311 (24.55639500), SSIM 0.70430040 (0.76289552)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00260656 (0.00227853), PSNR 25.83932877 (26.44394465), SSIM 0.73712683 (0.78015606)
Finished training epoch 62
Validate: MSE 0.00327342 (0.00470994), PSNR 24.84997749 (23.47198428), SSIM 0.68957686 (0.73411753)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00201408 (0.00226538), PSNR 26.95922661 (26.46951847), SSIM 0.79313940 (0.78023464)
Finished training epoch 63
Validate: MSE 0.00265715 (0.00236593), PSNR 25.75583076 (26.33519031), SSIM 0.71699601 (0.77890716)
Finished validation.
Starting training epoch 64
Epoch: 64, MSE 0.0022771

Validate: MSE 0.00263664 (0.00235884), PSNR 25.78949547 (26.34098812), SSIM 0.71776611 (0.77637940)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00232125 (0.00224609), PSNR 26.34277916 (26.50669416), SSIM 0.76738721 (0.78025807)
Finished training epoch 91
Validate: MSE 0.00293746 (0.00283101), PSNR 25.32028389 (25.56076411), SSIM 0.70937288 (0.77074111)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00229986 (0.00223082), PSNR 26.38298035 (26.53819458), SSIM 0.77126181 (0.78037055)
Finished training epoch 92
Validate: MSE 0.00282861 (0.00232398), PSNR 25.48427200 (26.39800982), SSIM 0.71365893 (0.77359045)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00206546 (0.00223662), PSNR 26.84983444 (26.52845619), SSIM 0.80548209 (0.78028634)
Finished training epoch 93
Validate: MSE 0.00289532 (0.00237245), PSNR 25.38303185 (26.32171534), SSIM 0.71859729 (0.78076752)
Finished validation.
Starting training epoch 94
Epoch: 94, MSE 0.0023897

Epoch: 120, MSE 0.00241466 (0.00222654), PSNR 26.17144585 (26.54756330), SSIM 0.77889067 (0.78040250)
Finished training epoch 120
Validate: MSE 0.00257631 (0.00228063), PSNR 25.89001274 (26.48180075), SSIM 0.71687174 (0.77868665)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00238725 (0.00221514), PSNR 26.22101402 (26.56842710), SSIM 0.78176600 (0.78041071)
Finished training epoch 121
Validate: MSE 0.00278683 (0.00332380), PSNR 25.54889107 (24.95264910), SSIM 0.71039665 (0.75996393)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00206750 (0.00221496), PSNR 26.84555054 (26.56639129), SSIM 0.78497255 (0.78056363)
Finished training epoch 122
Validate: MSE 0.00280368 (0.00243178), PSNR 25.52271843 (26.20074746), SSIM 0.71355200 (0.77201023)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00253584 (0.00222403), PSNR 25.95878601 (26.55358577), SSIM 0.76623207 (0.78022468)
Finished training epoch 123
Validate: MSE 0.00262935 (0.00226

Validate: MSE 0.00266610 (0.00225585), PSNR 25.74123764 (26.52335497), SSIM 0.71671474 (0.77832550)
Finished validation.
Starting training epoch 150
Epoch: 150, MSE 0.00230114 (0.00219977), PSNR 26.38057518 (26.59805822), SSIM 0.78502321 (0.78046071)
Finished training epoch 150
Validate: MSE 0.00278320 (0.00231347), PSNR 25.55455208 (26.42198155), SSIM 0.71817249 (0.77910192)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00227663 (0.00220182), PSNR 26.42706680 (26.59412563), SSIM 0.78782338 (0.78037429)
Finished training epoch 151
Validate: MSE 0.00312198 (0.00260569), PSNR 25.05570221 (25.92540736), SSIM 0.71191776 (0.77684805)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00204805 (0.00220033), PSNR 26.88660049 (26.59909568), SSIM 0.78465974 (0.78056887)
Finished training epoch 152
Validate: MSE 0.00269736 (0.00259136), PSNR 25.69061470 (25.93453150), SSIM 0.71183372 (0.76817371)
Finished validation.
Starting training epoch 153
Epoch: 153, MS

Epoch: 179, MSE 0.00212839 (0.00219241), PSNR 26.71949196 (26.61309919), SSIM 0.79436052 (0.78044538)
Finished training epoch 179
Validate: MSE 0.00261885 (0.00241697), PSNR 25.81888580 (26.23106549), SSIM 0.71351516 (0.77549713)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00261885 (0.00241697), PSNR 25.81888580 (26.23106552), SSIM 0.71351516 (0.77549713)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()